In [2]:
import pandas as pd
from pathlib import Path

In [3]:
train = Path('train')

In [4]:
eval_ = Path('eval')

In [5]:
test = Path('test')

In [6]:
# ใน train มี 7 พันไฟล์.  เอาที่เป็น ".xxx" ออก # โค้ดจาก https://colab.research.google.com/drive/1Ur-p2yQBf5JsUJbJJlZsm2VqNPY8gIkL
train_files = [f for f in train.iterdir() if not f.name.startswith('.')]+[f for f in eval_.iterdir() if not f.name.startswith('.')]
len(train_files)

4268

In [7]:
# ใน train มี 7 พันไฟล์.  เอาที่เป็น ".xxx" ออก
test_files = [f for f in test.iterdir() if not f.name.startswith('.')]
len(test_files)

483

In [8]:
list(set(train_files) & set(test_files)) # หาไฟล์ซ้ำ

[]

In [9]:
def readfile(path):
    _data = []
    with open(path,'r',encoding="utf-8-sig") as f:
        _temp = [i for i in f.read().split("\n\n") if i != '']
    for j in _temp:
        _temp_data = []
        for i in j.split("\n"):
            _t = i.strip().split("\t")
            _temp_data.append((_t[0],_t[1],_t[3]))
        _data.append(_temp_data)
    return _data

In [10]:
train_data = []

In [11]:
for file in train_files:
    train_data.extend(readfile(file))

In [12]:
len(train_data)

68930

In [13]:
test_data = []

In [14]:
for file in test_files:
    test_data.extend(readfile(file))

In [15]:
len(test_data)

5250

In [16]:
for i in train_data:
    if i in test_data:
        print(i)

[('การ', 'FX', 'B_CLS'), ('แข่งขัน', 'VV', 'I_CLS'), ('กระบวนการ', 'NN', 'I_CLS'), ('แก้', 'VV', 'I_CLS'), ('ปัญหา', 'NN', 'I_CLS'), ('ทาง', 'NN', 'I_CLS'), ('วิทยาศาสตร์', 'NN', 'E_CLS')]
[('นาย', 'NN', 'B_CLS'), ('บุญจง', 'NN', 'I_CLS'), ('_', 'PU', 'I_CLS'), ('วงศ์ไตรรัตน์', 'NN', 'I_CLS'), ('_', 'PU', 'I_CLS'), ('รัฐมนตรี', 'NN', 'I_CLS'), ('ช่วย', 'VV', 'I_CLS'), ('ว่าการ', 'VV', 'I_CLS'), ('กระทรวง', 'NN', 'I_CLS'), ('มหาดไทย', 'NN', 'E_CLS')]
[('นาย', 'NN', 'B_CLS'), ('ศุภชัย', 'NN', 'I_CLS'), ('_', 'PU', 'I_CLS'), ('โพธิ์สุ', 'NN', 'I_CLS'), ('_', 'PU', 'I_CLS'), ('รัฐมนตรี', 'NN', 'I_CLS'), ('ช่วย', 'VV', 'I_CLS'), ('ว่าการ', 'VV', 'I_CLS'), ('กระทรวง', 'NN', 'I_CLS'), ('เกษตร', 'NN', 'I_CLS'), ('และ', 'CC', 'I_CLS'), ('สหกรณ์', 'NN', 'E_CLS')]
[('นาย', 'NN', 'B_CLS'), ('โสภณ', 'NN', 'I_CLS'), ('_', 'PU', 'I_CLS'), ('ซารัมย์', 'NN', 'I_CLS'), ('_', 'PU', 'I_CLS'), ('รัฐมนตรี', 'NN', 'I_CLS'), ('ว่าการ', 'VV', 'I_CLS'), ('กระทรวง', 'NN', 'I_CLS'), ('คมนาคม', 'NN', 'E_CLS')]


In [17]:
def doc2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    # Features from current word
    features={
        'word.word': word,
        'word.isspace':word.isspace(),
        'postag':postag,
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        prevword = doc[i-1][0]
        postag1 = doc[i-1][1]
        features['word.prevword'] = prevword
        features['word.previsspace']=prevword.isspace()
        features['word.prepostag'] = postag1
        features['word.prevwordisdigit'] = prevword.isdigit()
    else:
        features['BOS'] = True # Special "Beginning of Sequence" tag
    # Features from next word
    if i < len(doc)-1:
        nextword = doc[i+1][0]
        postag1 = doc[i+1][1]
        features['word.nextword'] = nextword
        features['word.nextisspace']=nextword.isspace()
        features['word.nextpostag'] = postag1
        features['word.nextwordisdigit'] = nextword.isdigit()
    else:
        features['EOS'] = True # Special "End of Sequence" tag
    return features

def extract_features(doc):
    return [doc2features(doc, i) for i in range(len(doc))]

def get_labels(doc):
    return [tag for (token,postag,tag) in doc]

In [18]:
from tqdm import tqdm

In [19]:
X_data = [extract_features(doc) for doc in tqdm(train_data)]
y_data = [get_labels(doc) for doc in tqdm(train_data)]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68930/68930 [00:00<00:00, 123309.95it/s]


In [20]:
import pycrfsuite

In [21]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_data, y_data):
    try:
      trainer.append(xseq, yseq)
    except:
      pass

Wall time: 30.8 s


In [22]:
trainer.set_params({
    'c1': 0.1,   # coefficient for L1 penalty
    'c2': 0.1,  # coefficient for L2 penalty
    'max_iterations': 500,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [23]:
%%time
trainer.train('cls.crfsuite')

Wall time: 12min 46s


In [33]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from itertools import chain
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [34]:
X_test = [extract_features(doc) for doc in tqdm(test_data)]
y_test = [get_labels(doc) for doc in tqdm(test_data)]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5250/5250 [00:00<00:00, 240912.18it/s]


In [35]:
tagger = pycrfsuite.Tagger()
tagger.open('cls.crfsuite')

In [36]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

Wall time: 1.74 s


In [40]:
tagger.tag(X_test[0])

['B_CLS',
 'I_CLS',
 'I_CLS',
 'I_CLS',
 'I_CLS',
 'I_CLS',
 'I_CLS',
 'I_CLS',
 'I_CLS',
 'E_CLS']

In [37]:
labels = list(tagger.info().labels.keys())

In [38]:
from sklearn.metrics import f1_score

In [39]:
print(bio_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       B_CLS       0.90      0.94      0.92     16111
       E_CLS       0.90      0.94      0.92     15947
       I_CLS       0.99      0.97      0.98    169565

   micro avg       0.97      0.97      0.97    201623
   macro avg       0.93      0.95      0.94    201623
weighted avg       0.97      0.97      0.97    201623
 samples avg       0.94      0.94      0.94    201623



d:\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
